In [1]:
import csv
import pandas as pd
import numpy as np
import os
import arff
import re
import codecs
path_to_dataset_folder = '../DATAsets'
path_to_ready_csv = path_to_dataset_folder + '/DATA/CSVs'

In [11]:
#Display counts of processable files in dataset folder
txt_count=0
xlsx_count=0
csv_count=0
data_count=0
arff_count=0
for filename in os.listdir(path_to_dataset_folder):
    if filename.endswith('.txt'):
        txt_count+=1
    elif filename.endswith('.xls') or filename.endswith('.xlsx'):
        xlsx_count+=1
    elif filename.endswith('.data') or filename.endswith('.dat'):
        data_count+=1
    elif filename.endswith('.csv'):
        csv_count+=1
    elif filename.endswith('.arff'):
        arff_count+=1
print('CSV:' + str(csv_count))
print('txt:' + str(txt_count))
print('excel:' + str(xlsx_count))
print('data:' + str(data_count))
print('arff:' + str(arff_count))

CSV:0
txt:0
excel:0
data:0
arff:0


In [5]:
def checkTargetDir(full_name):
    writeDir = ''.join('/'.join(full_name.split('/')[0:-1]) + '/CSVs')
    if not os.path.isdir(writeDir):
        os.mkdir(writeDir)
    return writeDir

In [6]:
def tabAndWhite(full_name):
    print('Dealing with tabs and whitespaces in: ' + full_name.split('/')[-1])
    with codecs.open(full_name, 'r', 'utf-8') as curFile:
        data = curFile.readlines()
    data = [re.sub('\t', ' ',x.strip()) for x in data]    #Turning tabs to whitespace,stripping start/end.
    data = [re.sub(' +',' ',x) for x in data]    #Turning multiple whitespaces to length of one.
    data = '\n'.join(data)
    tempFile = ''.join('/'.join(full_name.split('/')[0:-1]) + '/v2_' + full_name.split('/')[-1])
    with codecs.open(tempFile, 'w', 'utf-8') as curFile:
        for row in data:
            curFile.write(row)
    return tempFile

In [7]:
def process_csv_txt_data(original_name):
    filename = original_name.split('/')[-1]
    print('Processing ' + filename)
    full_name = tabAndWhite(original_name)
    file_size = os.stat(full_name).st_size
    with codecs.open(full_name, 'r', 'utf-8') as curFile:
        #Dealing with small (if) and bigger size files (else) while detecting header.
        if file_size < 4100:
            header_present = csv.Sniffer().has_header(''.join(curFile.readlines(file_size - 50)))
        else:
            header_present = csv.Sniffer().has_header(''.join(curFile.readlines(4096)))
        curFile.seek(0)
        if header_present:
            print('Header present in file: ' + filename)
            df = pd.read_table(curFile,header = 0, sep = None)
        else:
            print('No header in file: ' + filename)
            df = pd.read_table(curFile,header = None, sep = None)
    os.remove(full_name)
    writeDir = checkTargetDir(full_name)
    df.to_csv(os.path.join(writeDir,'.'.join(filename.split('.')[0:-1]) +'.csv'),index =False)
    #print(df.head())    #For visual confirmation
    print('----------------------------------------------------------------------')

In [8]:
def process_excel(full_name):
    filename = full_name.split('/')[-1]
    print('Processing ' + filename)
    df = pd.read_excel(full_name)
    writeDir = checkTargetDir(full_name)
    df.to_csv(os.path.join(writeDir,'.'.join(filename.split('.')[0:-1]) +'.csv'),index =False)
    #print(df.head())    #For visual confirmation
    print('----------------------------------------------------------------------')

In [9]:
def process_arff(full_name):
    print('Processing ' + full_name.split('/')[-1])
    with codecs.open(full_name,encoding = 'utf-8') as curFile:
        df = arff.load(curFile)
    df = pd.DataFrame(df['data'])
    writeDir = checkTargetDir(full_name)
    df.to_csv(os.path.join(writeDir,'.'.join(filename.split('.')[0:-1]) +'.csv'),index =False)
    #print(df.head())    #For visual confirmation
    print('----------------------------------------------------------------------')

In [ ]:
#Creating "files" as a one time assignment of listdir so as to
#avoid processing later-created-temp files as well.
files = os.listdir(path_to_dataset_folder)
for filename in files:
    full_name = os.path.join(path_to_dataset_folder, filename)
    suf = filename.split('.')[-1]
    if suf in ["txt","csv","data","dat"]:
        process_csv_txt_data(full_name)
    elif suf in ["xls", "xlsx"]:
        process_excel(full_name)
    elif suf == "arff":
        process_arff(full_name)
    else:
        print('Unsupported file: ' + filename + ' ...ignoring...')

In [ ]:
#Creating City Files of weather attributes form 'Historical Hourly Weather Data'
#Files with attributes must be in a subfolder "Historical_Hourly_Weather_Data_for_processing" 
#while the file "city_attributes.csv" needs to be on the same dir as the folder 
#"Historical_Hourly_Weather_Data_for_processing"
process_path = os.path.join(path_to_dataset_folder + '/Historical_Hourly_Weather_Data_for_processing')
attr_list =[]
for file in os.listdir(process_path):
    attr_list.append(file.split('.')[0])
df = pd.read_table(os.path.join(path_to_dataset_folder,'city_attributes.csv'),sep=',')
city_names = []
for item in df.City:
    city_names.append(item)
for i in range(len(city_names)):
    print('Checking file: ' + city_names[i] +'.csv')
    df = pd.DataFrame(columns=attr_list)
    for file in os.listdir(process_path):
        temp = pd.read_table(os.path.join(process_path, file),sep=',')
        df.loc[:,file.split('.')[0]] = temp.loc[:,city_names[i]]
    df = df.dropna(how='all')    #Drop lines where all values are NaN.
    if df.isnull().sum().sum() < 500:    #Keep the files with less than 500 total NaN values.
        df = df.dropna(how='any')    #From the kept files, drop lines with even one NaN.
        df.to_csv(path_to_dataset_folder +'/processed_'+city_names[i]+'.csv',index= False)
        print('Pass')
    else:
        print("Too many NaN's, ignoring")

In [ ]:
#This is used to see how many lines of data exist in the processesed CSVs
for file in os.listdir(path_to_dataset_folder):
    if file.split('.')[-1] == 'csv':
        df = pd.read_table(path_to_dataset_folder + '/'+ file,sep=',')
        print('File: ' + file)
        print(len(df))
        print('---------------------------------------------')

In [ ]:
#Droping lines with even one NaN value in the "PRSA_Data_20130301-20170228" files 
#The folder "PRSA_Data_20130301-20170228" needs to be in the "DATAsets" dir, and the new
#files are generated in the same dir.
for file in os.listdir(path_to_dataset_folder + '/PRSA_Data_20130301-20170228):
    if file.split('.')[-1] == 'csv':
        df = pd.read_table(path_to_dataset_folder + '/'+ file,sep=',')
        df = df.dropna(how='any')
        df.to_csv(path_to_dataset_folder +'/processed_'+file,index= False)
        print('File: ' + file)
        print(len(df))
        print('---------------------------------------------')

In [ ]:
#Printing the head() and other info of the processed CSV files for visual inspection.
files = os.listdir(path_to_ready_csv)
for tempfile in files:
    df = pd.read_table(path_to_ready_csv + '/' + tempfile, header = 0, sep = ',')
    print('File: ' + tempfile + ' | Lines:' + str(len(df)))
    types = df.dtypes
    cardi = df.apply(pd.Series.nunique)
    df2=pd.DataFrame({'Types': types,
                      'Cardinality': cardi})
    print(df2)
    print('\n')
    print(df.head())
    print('---------------------------------------------')